<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpusdev/blob/main/part2_word_embeddings_v06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Vectors

Presentations [slides](intro-word-vectors-DAAD-2021-v22.pdf); Original [slides](word-vectors.pdf)

Word vectors (also known as 'word embeddings') are one of the most popular kinds of AI models. They are extremely useful in many domains. In essence, a word vector is a set of numbers that attempt to capture the meaning of a word. In typical implementations, each word is represented by a set of 200-300 numbers. In linear algebra, a one-dimensional array of numbers is known as a 'vector', hence these sets of numbers representing words' meanings are known as 'word vectors'.

Using neural networks, we can expose the computer to a large amount of text, and allow it to learn an appropriate set of numbers for each word it encounters. In this notebook, we will learn about the most famous of all word vector algorithms, `word2vec`, which was first described by Tomas Mikolov and his team in 2013:

* Tomas Mikolov, Ilya Sutskever, and others, ‘Distributed Representations of Words and Phrases and Their Compositionality’, in Advances in Neural Information Processing Systems 26, ed. by C. J. C. Burges and others (Curran Associates, Inc., 2013), pp. 3111–19 <http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf>
* Tomas Mikolov, Kai Chen, and others, ‘Efficient Estimation of Word Representations in Vector Space’, ArXiv:1301.3781 Cs, 2013 <http://arxiv.org/abs/1301.3781>.

In fact, `word2vec` is not a single algorithm, but rather a family of similar algorithms. In this session we will consider just the most famous `word2vec` algorithm, namely the `skip-gram model` trained using `negative sampling`.

## Applications of Word Vectors

Word vectors allow the computer to 'understand' language far more effectively. Rather than seeing each word as simply an arbitrarily different object, a computer using word vectors can analyse each word as a point in 200- or 300-dimenstional space. Words that are similar in meaning will have similar word vectors. And as we will see, the spaces between the word vectors are also significant: the words are arranged in patterns that represent their relationships to one another.

Accordingly, most AI systems that process language now include a word vector layer as part of their architecure. When the system encounters some text (e.g. when you speak to Siri or Alexa), your words are converted into word vectors, *and then* the computer examines what the text says and determines how it should respond.

In the Humanities, word vectors have become a popular modelling tool, because they allow researchers to perform sophisticated analysis on large corpora of text. Some examples include:

* [The Women Writers Vector Toolkit](https://wwp.northeastern.edu/lab/wwvt/index.html)
* William L. Hamilton, Jure Leskovec, and Dan Jurafsky, ‘Diachronic Word Embeddings Reveal Statistical Laws of Semantic Change’, ArXiv:1605.09096 [Cs], 2018 <http://arxiv.org/abs/1605.09096>.
* Ryan Heuser, 'Semantic Networks' <https://ryanheuser.org/word-vectors-4/>

## Training a `word2vec` model in Gensim

It is very easy to train a `word2vec` model in Gensim, which includes Mikolov's original `word2vec` code in its codebase.

In [1]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data
corpus = api.load('text8')
# api.info()

[==================================================] 100.0% 31.6/31.6MB downloaded


text8 is first 100MB from English wikipedia, 17 million words

The file is downloadable from
http://mattmahoney.net/dc/text8.zip


In [2]:
# Examine the corpus to see what is there
api.info("text8")

{'num_records': 1701,
 'record_format': 'list of str (tokens)',
 'file_size': 33182058,
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'license': 'not found',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'checksum': '68799af40b6bda07dfa47a32612e5364',
 'file_name': 'text8.gz',
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'parts': 1}

In [3]:
type(corpus)

text8.Dataset

In [4]:
data = [d for d in corpus]

In [5]:
len(data)

1701

In [6]:
len(data[0])

10000

In [7]:
print(data[0][:100])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing']


In [8]:
print(data[1][:100])

['reciprocity', 'qualitative', 'impairments', 'in', 'communication', 'as', 'manifested', 'by', 'at', 'least', 'one', 'of', 'the', 'following', 'delay', 'in', 'or', 'total', 'lack', 'of', 'the', 'development', 'of', 'spoken', 'language', 'not', 'accompanied', 'by', 'an', 'attempt', 'to', 'compensate', 'through', 'alternative', 'modes', 'of', 'communication', 'such', 'as', 'gesture', 'or', 'mime', 'in', 'individuals', 'with', 'adequate', 'speech', 'marked', 'impairment', 'in', 'the', 'ability', 'to', 'initiate', 'or', 'sustain', 'a', 'conversation', 'with', 'others', 'stereotyped', 'and', 'repetitive', 'use', 'of', 'language', 'or', 'idiosyncratic', 'language', 'lack', 'of', 'varied', 'spontaneous', 'make', 'believe', 'play', 'or', 'social', 'imitative', 'play', 'appropriate', 'to', 'developmental', 'level', 'restricted', 'repetitive', 'and', 'stereotyped', 'patterns', 'of', 'behavior', 'interests', 'and', 'activities', 'as', 'manifested', 'by', 'at', 'least', 'one']


### Step 1: Set hyperparameters and instantiate model

In [9]:
my_vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

model = Word2Vec(
    vector_size=my_vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

### Step 2: Fit model to corpus

In [10]:
# build a model
model.build_vocab(corpus)

In [11]:
# Train the model on the corpus
# model.train(sentences=corpus, epochs=5, total_examples=model.corpus_count)
model.train(corpus, epochs=5, total_examples=model.corpus_count)

(62531340, 85026035)

In [12]:
model.save("word2vec5e.model")

In [40]:
!zip word2vec5e.model.zip word2vec5e.model

  adding: word2vec5e.model (deflated 10%)


In [41]:
!rm word2vec.model

In [ ]:
!rm word2vec5e.model

In [13]:
!wget https://heibox.uni-heidelberg.de/f/b7625cd746534aacbefe/?dl=1
!mv index.html?dl=1 word2vec.model

--2024-06-06 10:17:42--  https://heibox.uni-heidelberg.de/f/b7625cd746534aacbefe/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/d69fb8d0-a96e-442f-92d9-bd4f851c6afa/word2vec.model [following]
--2024-06-06 10:17:44--  https://heibox.uni-heidelberg.de/seafhttp/files/d69fb8d0-a96e-442f-92d9-bd4f851c6afa/word2vec.model
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 59217897 (56M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  56.47M  7.75MB/s    in 11s     

2024-06-06 10:17:56 (5.14 MB/s) - ‘index.html?dl=1’ saved [59217897/59217897]



In [ ]:
# model = Word2Vec.load("word2vec.model")

In [14]:
model2 = Word2Vec.load("word2vec.model")

### Step 3: Extract word vectors from model

The fully trained model includes all of the weights used to predict the context words for each input word. If you are not planning on training the model further, these weights can be discarded, and you can just keep the weights for the word vectors.

In [15]:
word_vectors = model.wv
del model # Delete the whole model to free up the computer's RAM

In [17]:
word_vectors.save("word2vec.wordvectors5e")

In [39]:
!zip word2vec.wordvectors5e.zip word2vec.wordvectors5e

  adding: word2vec.wordvectors5e (deflated 12%)


In [42]:
!rm word2vec.wordvectors

In [43]:
!rm word2vec.wordvectors5e

In [19]:
!wget https://heibox.uni-heidelberg.de/f/e7725b1afab9456c8e16/?dl=1
!mv index.html?dl=1 word2vec.wordvectors

--2024-06-06 10:20:30--  https://heibox.uni-heidelberg.de/f/e7725b1afab9456c8e16/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/33242897-d5e7-4f76-8d47-4d7e83b65b4c/word2vec.wordvectors [following]
--2024-06-06 10:20:32--  https://heibox.uni-heidelberg.de/seafhttp/files/33242897-d5e7-4f76-8d47-4d7e83b65b4c/word2vec.wordvectors
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 30697035 (29M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  29.27M  7.94MB/s    in 4.9s    

2024-06-06 10:20:38 (6.00 MB/s) - ‘index.html?dl=1’ saved [30697035/30697035]



In [20]:
from gensim.models import KeyedVectors
wv2 = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [21]:
vector = wv2['computer']

In [22]:
vector

array([ 0.06077942,  0.1819358 , -0.05054147,  0.12852807,  0.01770034,
       -0.30997637, -0.24055079, -0.00272755, -0.6581456 , -0.18392453,
        0.06989374, -0.5224778 , -0.20864458, -0.10262147, -0.22059658,
        0.33618814,  0.04703446, -0.47088787, -0.1487342 , -0.6229217 ,
        0.42406246, -0.5138749 ,  0.06158032, -0.63825554, -0.07985036,
        0.16819875,  0.09989165, -0.17658606, -0.17862448,  0.29675245,
        0.4758706 ,  0.48978546,  0.9608675 , -0.22739977,  0.19066736,
        0.08846364,  0.07009576, -0.360089  , -0.06736434, -0.21808966,
       -0.2349182 , -0.25443017,  0.53662956,  0.2744204 ,  0.00105796,
       -0.12268312, -0.08814082,  0.00657543,  0.48798513, -0.11536589,
        0.2614905 ,  0.51717365, -0.04133837, -0.3994545 , -0.654159  ,
       -0.40764585,  0.7202135 , -0.2568953 ,  0.52592474,  0.11868921,
        0.1729699 ,  0.5800076 ,  0.46083173,  0.28301194,  0.58508384,
        0.46322995,  0.34925246,  0.02775881, -0.01378645,  0.07

### Step 4: Have a play with the model

There are several ways you can use word vectors. One of the most famous is to use them to compute analogies. The formula is:

<center><em>x</em> is to <em>small</em> as <em>biggest</em> is to <em>big</em></center>

$$x - vector('small') = vector('biggest') - vector('big')$$

$$\therefore x = vector('small') + vector('biggest') - vector('big')$$

In [23]:
# See the word vector for a particular word
vector = word_vectors['banana']
print(vector)

[-0.04113546  0.5782646   0.03752197  0.11038237  0.12040164  0.06393798
  0.23488076  0.20263442 -0.27701738 -0.10636657 -0.40006077  0.08516914
  0.15457365  0.02292606 -0.11117584 -0.02486759  0.3280512   0.05124469
 -0.21592592 -0.28669542  0.09007701 -0.01869808  0.94484055 -0.38864455
 -0.41059998  0.16247231 -0.18034849  0.46353462  0.05455324  0.03027178
 -0.16702564 -0.02473417 -0.08277375 -0.3991814  -0.04098587 -0.26466826
 -0.24117687 -0.06454886 -0.07376502 -0.59133756  0.29690832 -0.3835269
  0.1785365   0.33186638  0.3728563   0.05785524 -0.4090568   0.1563134
  0.58180356 -0.3121535  -0.36374938 -0.03683892  0.35559952  0.38890406
  0.00136656  0.16030996  0.30319896  0.35629863 -0.46200323 -0.1623705
  0.07147329  0.14452477 -0.49709588  0.18003334 -0.04173968  0.27392483
  0.1748616   0.12331246 -0.20455171 -0.37405783 -0.14353201  0.44899485
 -0.07031145 -0.2711735   0.36339694  0.4114501  -0.02345546  0.13470006
 -0.19512293 -0.29887038  0.16725068 -0.7159141  -0.11

In [28]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('university', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('college', 0.8330773711204529)
('urbana', 0.7925047278404236)
('institute', 0.7841007113456726)
('polytechnic', 0.7677932977676392)
('champaign', 0.7671976685523987)
('graduate', 0.7567100524902344)
('abet', 0.7500896453857422)
('rutgers', 0.7499120831489563)
('tsinghua', 0.7413315773010254)
('pomona', 0.7386504411697388)
('professor', 0.7352950572967529)
('yale', 0.7347093224525452)
('bloomington', 0.7311546206474304)
('vassar', 0.7274224162101746)
('emory', 0.7205010652542114)
('princeton', 0.7192938923835754)
('emeritus', 0.7183842658996582)
('mcmaster', 0.7180989384651184)
('iit', 0.7166054248809814)
('ryerson', 0.71498703956604)
('phd', 0.7126315832138062)
('suny', 0.7116893529891968)
('mellon', 0.7106456756591797)
('ucl', 0.7097887396812439)
('witwatersrand', 0.7096288800239563)


In [29]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('red', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('blue', 0.8395482301712036)
('yellow', 0.8381789922714233)
('green', 0.8018441200256348)
('white', 0.798026978969574)
('black', 0.7571162581443787)
('purple', 0.7408638596534729)
('orange', 0.6867749691009521)
('stripes', 0.6793258786201477)
('magenta', 0.6771360635757446)
('colored', 0.6733496785163879)
('dwarfs', 0.6624110341072083)
('striped', 0.6579605340957642)
('brindle', 0.6557778120040894)
('colors', 0.6523218750953674)
('jerseys', 0.652314305305481)
('sash', 0.651824414730072)
('talons', 0.6457521915435791)
('maroon', 0.6401049494743347)
('lorry', 0.6379421949386597)
('pink', 0.6352706551551819)
('cross', 0.6328311562538147)
('panther', 0.6315248608589172)
('pinstripes', 0.6256352066993713)
('cyan', 0.6227660775184631)
('hoist', 0.6203923225402832)


In [30]:
similar_words = word_vectors.most_similar('color', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('colour', 0.8844510912895203)
('colors', 0.8663884401321411)
('cyan', 0.7727386951446533)
('colours', 0.7582835555076599)
('colored', 0.7574570178985596)
('rgb', 0.7547838091850281)
('palettes', 0.7523167133331299)
('monochrome', 0.7264874577522278)
('hues', 0.7181781530380249)
('palette', 0.7132647037506104)
('pixel', 0.7079315185546875)
('subpixels', 0.7066963911056519)
('cmyk', 0.7060423493385315)
('monochromatic', 0.7030099034309387)
('pixels', 0.702581524848938)
('yellow', 0.69477379322052)
('backlight', 0.6921998262405396)
('srgb', 0.6870763897895813)
('magenta', 0.6866226196289062)
('violet', 0.6865247488021851)
('leds', 0.6864258646965027)
('flashes', 0.68503737449646)
('backlit', 0.6842491030693054)
('shade', 0.6821250915527344)
('hue', 0.6817468404769897)


In [31]:
similar_words = word_vectors.most_similar('colour', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('color', 0.8844510316848755)
('colors', 0.8149035573005676)
('colours', 0.7981035709381104)
('colored', 0.7869955897331238)
('hues', 0.7378509044647217)
('shade', 0.735605776309967)
('cyan', 0.7326695919036865)
('palettes', 0.7222548723220825)
('shades', 0.7220083475112915)
('coloration', 0.7185987234115601)
('glasses', 0.7155719995498657)
('palette', 0.7067922949790955)
('translucent', 0.7048293948173523)
('yellowish', 0.6972984671592712)
('brownish', 0.6971267461776733)
('monochromatic', 0.697028398513794)
('coloured', 0.6913246512413025)
('darker', 0.690552830696106)
('monochrome', 0.690521776676178)
('magenta', 0.6896342635154724)
('leds', 0.688128650188446)
('blue', 0.6877025961875916)
('texture', 0.6871539354324341)
('tint', 0.6854122281074524)
('rgb', 0.6840483546257019)


In [32]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('brussels', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('bruges', 0.752652108669281)
('aachen', 0.7368746995925903)
('rotterdam', 0.7358056902885437)
('ghent', 0.7338276505470276)
('strasbourg', 0.7311657071113586)
('paris', 0.7304145097732544)
('amsterdam', 0.7296232581138611)
('maastricht', 0.721036434173584)
('lausanne', 0.7093325257301331)
('bruxelles', 0.7073898315429688)
('brugge', 0.7053254246711731)
('belgium', 0.7017873525619507)
('bordeaux', 0.7001644372940063)
('commune', 0.6996681690216064)
('bayonne', 0.6923428177833557)
('frankfurt', 0.6915472149848938)
('antwerp', 0.6889127492904663)
('hamburg', 0.686873197555542)
('cologne', 0.6842682957649231)
('namur', 0.6825664639472961)
('fontainebleau', 0.6824367046356201)
('charleroi', 0.68219393491745)
('geneva', 0.6785435080528259)
('basel', 0.6772386431694031)
('leuven', 0.6751503944396973)


In [33]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('heidelberg', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('freiburg', 0.8657544255256653)
('grammatik', 0.8311512470245361)
('bingen', 0.8166913390159607)
('linz', 0.8163313269615173)
('halle', 0.8161386847496033)
('ttingen', 0.8032628297805786)
('ingolstadt', 0.7976961731910706)
('sorbonne', 0.7973642349243164)
('hofstra', 0.7935213446617126)
('zurich', 0.7911937236785889)
('breslau', 0.7900843024253845)
('breisgau', 0.787092387676239)
('jena', 0.7862498760223389)
('regensburg', 0.784187376499176)
('leiden', 0.7828952074050903)
('strassburg', 0.7826840877532959)
('graz', 0.7789398431777954)
('philosophie', 0.7770108580589294)
('kassel', 0.7769522070884705)
('logik', 0.77618807554245)
('wien', 0.7760013341903687)
('wissenschaft', 0.7758734822273254)
('allgemeine', 0.7757799029350281)
('laude', 0.7753399610519409)
('lucasian', 0.7716454863548279)


In [34]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('international', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('electrotechnical', 0.748532235622406)
('hydrographic', 0.7210329174995422)
('lome', 0.6874794363975525)
('nongovernmental', 0.6796085238456726)
('supranationalism', 0.6787280440330505)
('radiocommunications', 0.6779969930648804)
('telecommunication', 0.6728586554527283)
('ilo', 0.6677930951118469)
('intergovernmental', 0.664245069026947)
('paralympic', 0.6616351008415222)
('mesures', 0.6610217690467834)
('bipm', 0.6608805656433105)
('medarabtel', 0.6594502329826355)
('icsu', 0.6576040983200073)
('bretton', 0.6564810276031494)
('mercosur', 0.6543207168579102)
('itso', 0.652675211429596)
('usaid', 0.6490035057067871)
('cemac', 0.6465795040130615)
('interpol', 0.6456258296966553)
('sanctioning', 0.6455206871032715)
('undp', 0.6454935669898987)
('internacia', 0.6440893411636353)
('saarc', 0.6438179612159729)
('organization', 0.6430097222328186)


In [35]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('germany', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('austria', 0.8207393884658813)
('hungary', 0.7929275035858154)
('italy', 0.7581880688667297)
('berlin', 0.7465624213218689)
('dresden', 0.7416966557502747)
('belgium', 0.7401780486106873)
('switzerland', 0.7294812202453613)
('netherlands', 0.7286902070045471)
('bavaria', 0.7233483195304871)
('czechoslovakia', 0.7228786945343018)
('poland', 0.7186285853385925)
('alsace', 0.7180007696151733)
('denmark', 0.7150334715843201)
('gdr', 0.7140324711799622)
('weimar', 0.7132725715637207)
('nazi', 0.710624635219574)
('sudetenland', 0.7089345455169678)
('finland', 0.7079053521156311)
('romania', 0.7028300762176514)
('rhineland', 0.7011822462081909)
('german', 0.7003368735313416)
('russia', 0.6968211531639099)
('europe', 0.6950827836990356)
('reunified', 0.6938440203666687)
('munich', 0.6928016543388367)


In [36]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('strange', topn=25)
print('\n'.join([str(tup) for tup in similar_words]))

('mysterious', 0.6894012689590454)
('rogues', 0.6872665286064148)
('hilarious', 0.6867589354515076)
('madness', 0.6595258712768555)
('startling', 0.658959150314331)
('amazing', 0.6568843126296997)
('bizarre', 0.6557173728942871)
('dread', 0.6545179486274719)
('cheerful', 0.6487147212028503)
('unforgettable', 0.647528886795044)
('dreamer', 0.6474066376686096)
('fictionalised', 0.645953893661499)
('haunting', 0.6447542309761047)
('scary', 0.6433145403862)
('improbable', 0.6430540084838867)
('riddles', 0.6428862810134888)
('kitten', 0.6418792009353638)
('weird', 0.6412800550460815)
('fantastic', 0.6388643383979797)
('fantastical', 0.6370919346809387)
('campy', 0.636388897895813)
('fiendish', 0.6362330317497253)
('mischievous', 0.6355268359184265)
('archetypal', 0.6351879835128784)
('nightmarish', 0.634970486164093)


In [38]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['man'], positive=['king','woman'], topn=20)
print('\n'.join([str(tup) for tup in analogous_words]))

('queen', 0.6624085903167725)
('jadwiga', 0.6285076141357422)
('valois', 0.6215748190879822)
('throne', 0.6173731088638306)
('wenceslaus', 0.6108902096748352)
('sigismund', 0.6089454293251038)
('consort', 0.6088487505912781)
('matilda', 0.6069800853729248)
('boleslaus', 0.6048868894577026)
('haakon', 0.6039324998855591)
('emperor', 0.6024051904678345)
('dowager', 0.6020401120185852)
('olympias', 0.6019123196601868)
('daughter', 0.5997915267944336)
('prince', 0.5955933928489685)
('vii', 0.592556357383728)
('canute', 0.592311680316925)
('empress', 0.5903018712997437)
('concubine', 0.5897650718688965)
('aquitaine', 0.5892679691314697)


In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['king'], positive=['queen','man'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mother'], positive=['father','girl'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# x is to daughter as people is to person (plural + daughter)
analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
analogous_words = word_vectors.most_similar(negative=['soft'], positive=['hard','softer'])
# analogous_words = word_vectors.most_similar(negative=['soft'], positive=['hard','softest'])
print('\n'.join([str(tup) for tup in analogous_words]))

## Step 4A - optional. Using pre-trained models in Gensim

In many applications, you will simply want access to pre-trained word vectors (e.g. for plugging in to another model you are training). If you don't need the vectors to be tailored closely to your particular corpus, then you might like to use some pretrained models.

`word2vec` is not the only word embedding family of algorithms. Another, arguably even more powerful algorithm is the `FastText` algorithm, which Mikolov developed after moving to Facebook:

* Piotr Bojanowski and others, ‘Enriching Word Vectors with Subword Information’, ArXiv:1607.04606, 2017 <http://arxiv.org/abs/1607.04606>.

Instead of computing word vectors for each word, FastText splits each word into its constituent chunks. For example, 'cat' would be split into 'c', 'a', 't', 'ca', 'at' and 'cat', and 'burp' would be split into 'b', 'u', 'r', 'p', 'bu', 'ur', 'rp', 'bur', 'urp' and 'burp'. Then a vector is computer for each chunk that appears in the corpus. Each word is represented as the mean of all the chunks that make it up. FastText is able to learn very good word vectors because it can extract meaning from subword units, e.g. it can see that 'television', 'telegraph' and 'telepathy' all have 'tele' at the front, and can see that 'formality', 'criminality' and 'paucity' share subword units such as 'al' and 'ity'.

You can access many pretrained models using the Gensim downloader. Using the cells below, you can try out some of the different models available through Gensim. Along with `word2vec` and `FastText`, Gensim also supports `Glove` and `Doc2Vec` models.

**NB:** These trained models are very large, and will take a while to download. You may wish to download this notebook and execute the cells below on your own machine, in case Google kicks you out of the Colab environment.

In [ ]:
# See what models are on offer
print(list(api.info()['models'].keys()))

In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# 300-dimensional word vectors trained on a huge dataset from Google News
google_news_w2v = api.load('word2vec-google-news-300')

In [ ]:
# x is to Kenya as Canberra is to Australia
google_news_w2v.most_similar(negative=['australia'],positive=['kenya','canberra'], topn=10)

In [ ]:
# Optional! THIS WILL TAKE SOME TIME!!!
# Facebook's own FastText vectors, trained on Wikipedia
wikipedia_fasttext = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
# x is to Wharton as London is to Dickens
wikipedia_fasttext.most_similar(negative=['dickens'],positive=['wharton','london'], topn=10)

In [ ]:
# try your own examples...

## Step 5. Training models in other languages (Armenian, Georgian, Ukrainian ...)

Now let's train a model on our own corpus. You can try to create Armenian / Ukrainian / Russian / German / French word vector model.

Armenian, Ukrainian, Russian plaintext wikipedias are available at:
https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-2735#

We can use the same size ~15 Million Words, (200MB) of these corpora to make sure the model is built within a reasonable time; however, when you have more time, you can try a more complete corpus.

It is possible to download/unzip/upload the files manually, however, it is faster to automatically load them directly from Heidelberg server space https://heibox.uni-heidelberg.de/d/42a07c9e95774e099a81/  into your colab environment. To do this run the cells 5a or 5b, or 5c, or 5d, or 5e -- depending on which language you prefer -- to download the data. Then skip other language and continue with Step5 -- building the model for your preferred language from the data you downloaded.


In [ ]:
!rm /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt
!rm index.html?dl=1

rm: cannot remove '/usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt': No such file or directory
rm: cannot remove 'index.html?dl=1': No such file or directory


### Either: 5a Armenian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/ffe527ed3d1e4b4cb43a/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5b Ukrainian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/676bef3a6db8482e9665/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5c German

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/ca0a085347c34563a6e6/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5d French

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/5cdb485efd4046f2a457/?dl=1

In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Or: 5f Georgian

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/fa3509d869b949459f91/?dl=1

Lemmatized corpus

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/b381c458ad5e4773a77d/?dl=1

In [ ]:
!cp index.html?dl=1 wiki_geo_lem.txt

In [ ]:
!head --lines=10 wiki_geo_lem.txt

In [ ]:
FIn = open("wiki_geo_lem.txt", 'r')
FOut = open('wiki_geo_l.txt', 'w')

i = 0
for SLine in FIn:
  i = i + 1
  SLine = SLine.strip() + ' '
  if i % 10000 == 0:
    SLine += '\n'

  FOut.write(SLine)

FOut.flush()
FOut.close()


In [ ]:
!head --lines=10 wiki_geo_l.txt

In [ ]:
!wc wiki_geo_lem.txt

 18054750  18054749 329202500 wiki_geo_lem.txt


In [ ]:
!cp index.html?dl=1 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

Alternatively, copy lemmatized corpus

In [ ]:
!cp wiki_geo_l.txt /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

In [ ]:
!head --lines=15 /usr/local/lib/python3.7/dist-packages/gensim/test/test_data/myOwnLangText8.txt

### Stage6: Training own model

optional: to clean disk space, we remove the downloaded file

In [ ]:
# optional: to clean disk space, we remove the downloaded file
!rm index.html\?dl\=1

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('myOwnLangText8.txt')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
import gensim.models
corpusOwn = MyCorpus()

optional: examining what is in the corpus after standard preprocessing

In [ ]:
# Optional: Examining our corpus format
type(corpusOwn)

__main__.MyCorpus

In [ ]:
dataOwn = [d for d in corpusOwn]

In [ ]:
print(len(dataOwn))
print(dataOwn[0])
print(len(dataOwn[0]))
print(dataOwn[4])
print(len(dataOwn[4]))
print(dataOwn[5])
print(len(dataOwn[5]))

... Initialising global parameters for our modelL vector size, collocation window, skip-grams, negative sampling....

In [ ]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data

In [ ]:
vector_size = 100 # Dimensionality of the word vectors
window = 5 # How many words either side? (5 = 5 context words either side, i.e. 10 context words in total)
use_skip_gram = 1 # If you set this to 0, then it will create a 'continuous bag of words' model instead
use_softmax = 0 # If you set this to 1, then hierarchical softmax will be used instead of negative sampling
negative_samples = 5 # How many incorrect answers to generate per correct answer when negative sampling

modelOwn = Word2Vec(
    size=vector_size,
    window=window,
    sg=use_skip_gram,
    hs=use_softmax,
    negative=negative_samples
)

... this cell may run for ~2 min or so...

In [ ]:
modelOwn.build_vocab(corpusOwn)

THIS MAY TAKE LONG!!! ... training the model may take 9 to 15 minutes... (just grab a cup of coffee or a sandwich while you are waiting... You may try chaning the number of epochs; if the number is lower the training is faster, but the quality may be lower...

In [ ]:
modelOwn.train(sentences=corpusOwn, epochs=5, total_examples=modelOwn.corpus_count)

(59378395, 70859245)

Now we copy word vectors and remove the model from memory (just to free up the resources...)

In [ ]:
word_vectors_own = modelOwn.wv

In [ ]:
del modelOwn

Now we can examine the output of our word embeddings

### German examples

In [ ]:
# DE examples
# See the word vector for a particular word
vector = word_vectors['banane']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('universität', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('rot', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('brüssel', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('deutschland', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# See which words are closest to a given word in the vector space
similar_words = word_vectors.most_similar('komisch', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mann'], positive=['könig','frau'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['könig'], positive=['königin','mann'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mutter'], positive=['vater','tochter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# E.g. x is to queen as man is to king => x = v('queen') + v('man') - v('king')
analogous_words = word_vectors.most_similar(negative=['mutter'], positive=['vater','mädchen'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# Compute analogous words
# x is to daughter as people is to person (plural + daughter)
analogous_words = word_vectors.most_similar(negative=['person'], positive=['menschen','tochter'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
analogous_words = word_vectors.most_similar(negative=['weich'], positive=['hart','weichste'])
print('\n'.join([str(tup) for tup in analogous_words]))

### Georgian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['სამყარო']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('არსება', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# blue
similar_words = word_vectors_own.most_similar('ლურჯი', topn=30)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# france
similar_words = word_vectors_own.most_similar('საფრანგეთი', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['კაცი'], positive=['მეფე','ქალი'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′)
analogous_words = word_vectors_own.most_similar(negative=['პირი'], positive=['ხალხი','ქალიშვილი'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])

### Armenian examples

In [ ]:
# See the word vector for a particular word -- 'world'
vector = word_vectors_own['աշխարհը']
print(vector)

In [ ]:
# See which words are closest to a given word in the vector space = 'world'
similar_words = word_vectors_own.most_similar('աշխարհը', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# blue
similar_words = word_vectors_own.most_similar('կապույտ', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# france
similar_words = word_vectors_own.most_similar('ֆրանսիա', topn=10)
print('\n'.join([str(tup) for tup in similar_words]))

In [ ]:
# x is to king as woman is to man
analogous_words = word_vectors_own.most_similar(negative=['մարդ'], positive=['թագավոր','կին'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# x is to king as woman is to man
# x = small + biggest - big
# 𝑥−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′𝑠𝑚𝑎𝑙𝑙′)+𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔𝑔𝑒𝑠𝑡′)−𝑣𝑒𝑐𝑡𝑜𝑟(′𝑏𝑖𝑔′)
# ∴ 𝑥=𝑣𝑒𝑐𝑡𝑜𝑟(′փոքր′)+𝑣𝑒𝑐𝑡𝑜𝑟(′ամենամեծը′)−𝑣𝑒𝑐𝑡𝑜𝑟(′մեծ′)
analogous_words = word_vectors_own.most_similar(negative=['մեծ'], positive=['ամենամեծը','փոքր'])
print('\n'.join([str(tup) for tup in analogous_words]))

In [ ]:
# try your own examples... (also -- morphology?)
# # x is to daughter as people is to person (plural + daughter)
# analogous_words = word_vectors.most_similar(negative=['person'], positive=['people','daughter'])